# Problema das Mochilas Multiplas

<div style="text-align:justify">
O problema das mochilas multiplas (em <i>inglês</i>, Multi-Knapsack Problem) é um problema de optimização combinatória. O nome dá-se devido ao modelo de uma situação em que é necessário preencher <i>n</i> mochilas com objetos de diferentes pesos e valores. O objetivo é que se preencha as mochilas com o maior valor possível, não ultrapassando o peso máximo de cada uma.
</div>

## Variáveis

Seja:<br>
- C<sub>j</sub> = Capacidade da j-ésima mochila
- v<sub>i</sub> = Valor do <i>i-ésimo</i> objeto
- w<sub>i</sub> = Peso do <i>i-ésimo</i> objeto
- x<sub>ij</sub> = x<sub>ij</sub> &isin; {0, 1} &frasl; x<sub>i</sub> &sub; Mochila<sub>j</sub>

## Função Objetivo
<br>
Maximizar o valor dos itens escolhidos.

$$ max \sum_{i=0}^{n} \sum_{j=0}^{m} v_i.x_{ij} $$

## Restrições

O mesmo item deve estar contigo em no máximo uma mochila.

$$ \sum_{j=0}^{m} x_{ij} \leq 1, \forall i=0 ... n $$ 

O peso total dos itens escolhidos não pode exceder a capacidade de cada mochila.

$$ \sum_{i=0}^{n} w_i.x_{ij} \leq C_j, \forall j=0 ... m $$

## Implementação

<div style="text-align:justify">
Vamos implementar a solução modelada utilizando a Gurobi API. Para isso, vamos criar dados sintéticos que representem o peso e o valor dos objetos a serem adicionados na mochila.
</div>

#### Carregando Dependências

In [1]:
import gurobipy, pandas

#### Dados

In [2]:
capacities = [600, 375]

df = pandas.read_csv('datasets/knapsack.csv')
weights = df['weights'].values
values = df['values'].values

#### Lista de Itens

In [3]:
items = []
for i in range(len(weights)):
    items.append('item_{}'.format(i))

print(items)

['item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6', 'item_7', 'item_8', 'item_9', 'item_10', 'item_11', 'item_12', 'item_13', 'item_14']


#### Lista de Mochilas

In [4]:
knaps = []
for i in range(len(capacities)):
    knaps.append('knap_{}'.format(i))

print(knaps)

['knap_0', 'knap_1']


#### Capacidades

In [5]:
C = {}
for i, value in enumerate(knaps):
    C[value] = capacities[i]

C

{'knap_0': 600, 'knap_1': 375}

#### Dicionário de Pesos

In [6]:
w = {}
for idx, value in enumerate(weights):
    w[items[idx]] = value

print(w)

{'item_0': 65, 'item_1': 94, 'item_2': 119, 'item_3': 59, 'item_4': 149, 'item_5': 114, 'item_6': 57, 'item_7': 136, 'item_8': 100, 'item_9': 150, 'item_10': 122, 'item_11': 117, 'item_12': 120, 'item_13': 130, 'item_14': 133}


#### Dicionário de Valores

In [7]:
v = {}
for idx, value in enumerate(values):
    v[items[idx]] = value

print(v)

{'item_0': 455, 'item_1': 691, 'item_2': 833, 'item_3': 425, 'item_4': 1064, 'item_5': 758, 'item_6': 419, 'item_7': 914, 'item_8': 651, 'item_9': 966, 'item_10': 828, 'item_11': 827, 'item_12': 857, 'item_13': 837, 'item_14': 894}


#### Modelagem

In [8]:
model = gurobipy.Model()

# decision variables
x = model.addVars(items, knaps, vtype=gurobipy.GRB.BINARY)

# objective to optimize
model.setObjective(
    gurobipy.quicksum(v[i] * x[i,j] for i in items for j in knaps),
    sense=gurobipy.GRB.MAXIMIZE
)

# constraint 1
c1 = model.addConstrs(
    gurobipy.quicksum(x[i,j] for j in knaps) <= 1 for i in items
)

# constraint 2
c2 = model.addConstrs(
    gurobipy.quicksum(w[i] * x[i,j] for i in items) <= C[j] for j in knaps
)

model.optimize()

Restricted license - for non-production use only - expires 2022-01-13
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 30 columns and 60 nonzeros
Model fingerprint: 0xbd5c0cec
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [4e+02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 6210.0000000
Presolve time: 0.00s
Presolved: 17 rows, 30 columns, 60 nonzeros
Variable types: 0 continuous, 30 integer (30 binary)

Root relaxation: objective 6.889692e+03, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6889.69173    0    3 6210.00000 6889.69173  10.9%     -    0s
H    0     0                    

#### Itens Selecionados

In [9]:
approved = []
for j in knaps:
    print(j)
    for i in items:
        if round(x[i,j].X) == 1:
            print(i, end=' ')
    print('\n')

knap_0
item_1 item_3 item_4 item_6 item_10 item_11 

knap_1
item_2 item_7 item_12 



#### Valor Acumulado

In [10]:
print('Valor Objetivo: {}'.format(model.objVal))

Valor Objetivo: 6858.0


#### Capacidade Utilizada

In [11]:
for j in knaps:
    print('Capacidade Utilizada {}: {}'.format(j, C[j] - c2[j].Slack))

Capacidade Utilizada knap_0: 598.0
Capacidade Utilizada knap_1: 375.0
